In [ ]:
#Import necessary modules and packages
%matplotlib qt5
import numpy as np
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm_notebook

# Defining functions

In [ ]:
# MOUSE SELECT FUNCTION
def select_point(event, x, y, flags, params):
    global point_selected, points, point, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        points.append([x,y])
        point_selected = True
        
# SHOW FIRST FRAME OF VIDEO        
def get_frame(filename,index):
    counter=0
    video=cv2.VideoCapture(filename)
    while video.isOpened():
        rete,frame=video.read()
        if rete:
            if counter==index:
                return frame
            counter +=1
        else:
            break
    video.release()
    return None 

def count_frames_manual(video):
    # initialize the total number of frames read
    total = 0
    # loop over the frames of the video
    video=cv2.VideoCapture(video)
    while True:
        # grab the current frame
        (grabbed, frame) = video.read()

        # check to see if we have reached the end of the video
        if not grabbed:
            break
        # increment the total number of frames read
        total += 1
    # return the total number of frames in the video file
    return total

# Motion tracking

In [ ]:
# Ask for user input
val = input("Track the motions in x- or y-direction?: ")

if val == 'x':
    print('Place sensors from top to bottom on the structure, and press ESC when ready!')
else:
    print('Place sensors from top to bottom on the structure, and press ESC when ready!')  

# Video to Track
file = "filename"

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)

points = []
point_selected = False 
frame = get_frame(file,0) 

while True:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if point_selected == True:
        cv2.circle(frame, point, 5, (0,0,255), 2)
       
    cv2.imshow("Frame",frame)   
    
    key = cv2.waitKey(1)
    if key == 27:
        break  
        
old_points = np.array(points, dtype=np.float32)    
cv2.waitKey(1)

# Motion tracking
cap = cv2.VideoCapture(file)

# Details about video
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Count frames
total = count_frames_manual(file)

print("Total number of frames: %i" %total)
print("fps: %i" %fps)

#CREATE OLD FRAME
_, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#LUKAS KANADE PARAMS
lk_params = dict(winSize = (30,30), # endre størrelse på søkerom 
                maxLevel = 8, # Number of pyramid levels
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)

#Empty variables to save tracking data
count = 0
x_values = np.zeros([len(old_points),total-1]) 
y_values = np.zeros([len(old_points),total-1])

pbar = tqdm_notebook(total = total)

while True:
    ret, frame = cap.read()
    
    if not ret:
        print('Video Finished')
        break 
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if point_selected == True:
        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_params)
        old_gray = gray_frame.copy()
        old_points = new_points
        
        for i in range(len(new_points)):
            x, y = new_points[i].ravel()
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
            
            #Saving the values
            x_values[[i],[count]] = x
            y_values[[i],[count]] = y
      
    count = count + 1   
        
    cv2.imshow("Frame", frame)
    pbar.update(1)

    key = cv2.waitKey(1)
    
    if key == 27:
        break
    
pbar.close()            
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

np.savetxt('x_values.csv', x_values, delimiter=',')
np.savetxt('y_values.csv', y_values, delimiter=',')
np.savetxt('points.csv', points, delimiter=',')